# Scraping 20k Movies in IMDB

In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


## Import Modules

In [5]:
pip install pandas

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.1.2-cp311-cp311-win_amd64.whl.metadata (59 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached numpy-2.1.2-cp311-cp311-win_amd64.whl (12.9 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tqdm

  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
import re
from tqdm import tqdm  # Import tqdm for progress bar
from bs4 import BeautifulSoup
import json 

## WebDriver

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Base class for handling common WebDriver functionalities
class BaseScraper:
    def __init__(self):
        self.driver = self.init_driver()

    def init_driver(self):
        service = Service()  # Initialize the service
        options = webdriver.ChromeOptions()
        
        # Set Chrome options to reduce memory usage
        options.headless = True  # Enable headless mode
        options.add_argument('--disable-extensions')
        options.add_argument('--disable-gpu')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument("--no-sandbox")  # Use this if you encounter issues

        driver = webdriver.Chrome(service=service, options=options)
        return driver

    def close_driver(self):
        self.driver.quit()


## Scraping Movie Details

In [24]:
import json
import pickle
import re
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class MoviesScraper(BaseScraper):
    def __init__(self, clicks=200, batch_size=3, save_file='movies_data.json'):
        super().__init__()
        self.clicks = clicks
        self.batch_size = batch_size
        self.save_file = save_file
        # self.cookie_file = cookie_file  # File for saving cookies
        self.movie_data = []
        
        # self.load_cookies()  # Load cookies on startup

    # # Method to restart the driver
    # def restart_driver(self):
    #     self.close_driver()
    #     self.driver = self.init_driver()  # Reinitialize the driver
    #     self.load_cookies()  # Load cookies

    # def load_cookies(self):
    #     try:
    #         # Điều hướng đến đúng URL trước khi nạp cookies
    #         url = 'https://www.imdb.com/search/title/?title_type=feature'
    #         self.driver.get(url)

    #         with open(self.cookie_file, 'rb') as file:
    #             cookies = pickle.load(file)
    #             for cookie in cookies:
    #                 # Chỉ nạp cookies có domain khớp với tên miền của trang hiện tại
    #                 if 'domain' in cookie and cookie['domain'] in url:
    #                     self.driver.add_cookie(cookie)
    #                 else:
    #                     print(f"Skipping cookie with domain {cookie.get('domain')}")
    #         print(f"Cookies loaded from {self.cookie_file}")
    #     except FileNotFoundError:
    #         print("No cookies file found, starting fresh.")
    #     except Exception as e:
    #         print(f"Error loading cookies: {e}")


    # # Load cookies
    # def load_cookies(self):
    #     try:
    #         with open(self.cookie_file, 'rb') as file:
    #             cookies = pickle.load(file)
    #             for cookie in cookies:
    #                 self.driver.add_cookie(cookie)
    #         print(f"Cookies loaded from {self.cookie_file}")
    #     except FileNotFoundError:
    #         print("No cookies file found, starting fresh.")

    def fetch_movies(self):
        url = 'https://www.imdb.com/search/title/?title_type=feature'
        self.driver.get(url)

        total_batches = self.clicks // self.batch_size  # Calculate total batches
        remaining_clicks = self.clicks % self.batch_size  # Remaining clicks after batches

        # Process in batches
        for batch in range(total_batches):
            print(f"Processing batch {batch + 1}/{total_batches}...")
            self._process_batch(self.batch_size)
            self.save_partial_data(batch + 1)  # Save data after each batch
            # self.save_cookies()  # Save cookies after each batch

        # Process any remaining clicks
        if remaining_clicks > 0:
            print(f"Processing remaining {remaining_clicks} clicks...")
            self._process_batch(remaining_clicks)
            self.save_partial_data(total_batches + 1)  # Save data after remaining clicks
            # self.save_cookies()

        self.close_driver()

    # def _process_batch(self, num_clicks):
    #     """Process a batch with num_clicks of "see more" clicks."""
    #     with tqdm(total=num_clicks, desc='Loading movies') as pbar:
    #         for _ in range(num_clicks):
    #             soup = self.click_see_more_button()
    #             pbar.update(1)

    #     wait_time = self._calculate_wait_time(num_clicks)
    #     time.sleep(wait_time)

    #     # After each batch, extract movie data
    #     html = self.driver.page_source
    #     soup = BeautifulSoup(html, 'html.parser')
    #     movies = soup.select('div.sc-59c7dc1-2')
    #     self.extract_movie_data(movies)

    def _process_batch(self, num_clicks):
        """Process a batch with num_clicks of 'see more' clicks."""
        initial_html = self.driver.page_source
        initial_soup = BeautifulSoup(initial_html, 'html.parser')
        initial_movies = initial_soup.select('div.sc-59c7dc1-2')  # Saving first element

        with tqdm(total=num_clicks, desc='Loading movies') as pbar:
            for _ in range(num_clicks):
                soup = self.click_see_more_button()
                pbar.update(1)

        wait_time = self._calculate_wait_time(num_clicks)
        time.sleep(wait_time)

        # After pressing the 'see more' button 
        final_html = self.driver.page_source
        final_soup = BeautifulSoup(final_html, 'html.parser')
        final_movies = final_soup.select('div.sc-59c7dc1-2')

        # After each batch, extract movie data (just extract new data)
        new_movies = final_movies[len(initial_movies):]
        self.extract_movie_data(new_movies)


    def click_see_more_button(self):
        try:
            initial_elements = self.driver.find_elements(By.CLASS_NAME, 'ipc-title')
            initial_count = len(initial_elements)

            see_more_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '50 more')]"))
            )
            self.driver.execute_script("arguments[0].scrollIntoView(); arguments[0].click();", see_more_button)

            for _ in range(5):
                current_elements = self.driver.find_elements(By.CLASS_NAME, 'ipc-title')
                current_count = len(current_elements)

                if current_count > initial_count:
                    break
                time.sleep(1)

            return BeautifulSoup(self.driver.page_source, 'html.parser')

        except Exception as e:
            print(f"Error occurred: {e}")
            return None

    def save_partial_data(self, batch_num):
        """Save temporary data after each batch."""
        temp_file_name = f"movies_data_batch_{batch_num}.json"
        with open(temp_file_name, 'w', encoding='utf-8') as file:
            json.dump(self.movie_data, file, ensure_ascii=False, indent=4)
        print(f"Data for batch {batch_num} saved to {temp_file_name}")

    def extract_movie_data(self, movies):
        existing_movie_ids = {movie['Movie ID'] for movie in self.movie_data}  # Set of existing movie IDs

        for movie in movies:
            title_tag = movie.select_one('h3.ipc-title__text')
            link_tag = movie.select_one('a.ipc-title-link-wrapper')

            title = title_tag.text.strip() if title_tag else 'N/A'
            link = link_tag['href'] if link_tag else None

            if link_tag:
                movie_id = link.split('/title/')[1].split('/')[0]
            else:
                movie_id = 'N/A'

            title = re.sub(r'^\d+\.\s*', '', title)

            if movie_id not in existing_movie_ids:  # Only add if movie_id is not already in the list
                self.movie_data.append({
                    'Movie ID': movie_id,
                    'Title': title,
                })

    def _calculate_wait_time(self, clicks):
        base_wait_time = 5
        growth_factor = 1.2
        additional_wait_time = base_wait_time * (growth_factor ** (clicks // 10))
        return base_wait_time + additional_wait_time


In [ ]:
import json
import re
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class MoviesScraper(BaseScraper):
    def __init__(self, clicks=200, batch_size=2, save_file='movies_data.json'):
        super().__init__()
        self.clicks = clicks
        self.batch_size = batch_size
        self.save_file = save_file
        self.movie_data = []

    # Method to restart the driver
    def restart_driver(self):
        self.close_driver()
        self.driver = self.init_driver()  # Reinitialize the driver

    def fetch_movies(self):
        url = 'https://www.imdb.com/search/title/?title_type=feature'
        self.driver.get(url)

        total_batches = self.clicks // self.batch_size  # Calculate total batches
        remaining_clicks = self.clicks % self.batch_size  # Remaining clicks after batches

        # Process in batches
        for batch in range(total_batches):
            print(f"Processing batch {batch + 1}/{total_batches}...")
            self._process_batch(self.batch_size)
            self.save_partial_data(batch + 1)  # Save data after each batch

        # Process any remaining clicks
        if remaining_clicks > 0:
            print(f"Processing remaining {remaining_clicks} clicks...")
            self._process_batch(remaining_clicks)
            self.save_partial_data(total_batches + 1)  # Save data after remaining clicks

        self.close_driver()

    def _process_batch(self, num_clicks):
        """Process a batch with num_clicks of 'see more' clicks."""
        initial_html = self.driver.page_source
        initial_soup = BeautifulSoup(initial_html, 'html.parser')
        initial_movies = initial_soup.select('div.sc-59c7dc1-2')  # Saving first element

        with tqdm(total=num_clicks, desc='Loading movies') as pbar:
            for _ in range(num_clicks):
                soup = self.click_see_more_button()
                pbar.update(1)

        wait_time = self._calculate_wait_time(num_clicks)
        time.sleep(wait_time)

        # After pressing the 'see more' button
        final_html = self.driver.page_source
        final_soup = BeautifulSoup(final_html, 'html.parser')
        final_movies = final_soup.select('div.sc-59c7dc1-2')

        # After each batch, extract movie data (just extract new data)
        new_movies = final_movies[len(initial_movies):]
        self.extract_movie_data(new_movies)

    def click_see_more_button(self):
        try:
            initial_elements = self.driver.find_elements(By.CLASS_NAME, 'ipc-title')
            initial_count = len(initial_elements)

            see_more_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '50 more')]"))
            )
            self.driver.execute_script("arguments[0].scrollIntoView(); arguments[0].click();", see_more_button)

            for _ in range(5):
                current_elements = self.driver.find_elements(By.CLASS_NAME, 'ipc-title')
                current_count = len(current_elements)

                if current_count > initial_count:
                    break
                time.sleep(1)

            return BeautifulSoup(self.driver.page_source, 'html.parser')

        except Exception as e:
            print(f"Error occurred: {e}")
            return None

    def save_partial_data(self, batch_num):
        """Save temporary data after each batch."""
        temp_file_name = f"movies_data_batch_{batch_num}.json"
        with open(temp_file_name, 'w', encoding='utf-8') as file:
            json.dump(self.movie_data, file, ensure_ascii=False, indent=4)
        print(f"Data for batch {batch_num} saved to {temp_file_name}")

    def extract_movie_data(self, movies):
        existing_movie_ids = {movie['Movie ID'] for movie in self.movie_data}  # Set of existing movie IDs

        for movie in movies:
            title_tag = movie.select_one('h3.ipc-title__text')
            link_tag = movie.select_one('a.ipc-title-link-wrapper')

            title = title_tag.text.strip() if title_tag else 'N/A'
            link = link_tag['href'] if link_tag else None

            if link_tag:
                movie_id = link.split('/title/')[1].split('/')[0]
            else:
                movie_id = 'N/A'

            title = re.sub(r'^\d+\.\s*', '', title)

            if movie_id not in existing_movie_ids:  # Only add if movie_id is not already in the list
                self.movie_data.append({
                    'Movie ID': movie_id,
                    'Title': title,
                })

    def _calculate_wait_time(self, clicks):
        base_wait_time = 5
        growth_factor = 1.2
        additional_wait_time = base_wait_time * (growth_factor ** (clicks // 10))
        return base_wait_time + additional_wait_time


### Test

In [25]:
scraper = MoviesScraper(clicks=10)
scraper.fetch_movies()

Processing batch 1/3...


Loading movies: 100%|██████████| 3/3 [00:05<00:00,  1.81s/it]


Data for batch 1 saved to movies_data_batch_1.json
Processing batch 2/3...


Loading movies: 100%|██████████| 3/3 [00:05<00:00,  1.74s/it]


Data for batch 2 saved to movies_data_batch_2.json
Processing batch 3/3...


Loading movies: 100%|██████████| 3/3 [00:05<00:00,  1.93s/it]


Data for batch 3 saved to movies_data_batch_3.json
Processing remaining 1 clicks...


Loading movies: 100%|██████████| 1/1 [00:02<00:00,  2.56s/it]


Data for batch 4 saved to movies_data_batch_4.json


In [4]:
import os
import json

def check_size(file_name):
    # Đọc file JSON
    with open(os.path.join(os.getcwd(), file_name), 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Kiểm tra kiểu dữ liệu và kích thước
    if isinstance(data, list):
        print(f"Số lượng phần tử trong danh sách: {len(data)}")

        # Nếu phần tử là từ điển, kiểm tra thêm về trùng lặp và dữ liệu thiếu
        if data:
            first_element = data[0]
            print(f"Kích thước của phần tử đầu tiên: {len(first_element)} thuộc tính")

            # Kiểm tra trùng lặp (theo Movie ID)
            movie_ids = [item.get('Movie ID') for item in data if 'Movie ID' in item]
            duplicates = {movie_id for movie_id in movie_ids if movie_ids.count(movie_id) > 1}
            if duplicates:
                print(f"Các Movie ID trùng lặp: {duplicates}")
            else:
                print("Không có Movie ID trùng lặp.")

            # Kiểm tra dữ liệu thiếu
            missing_values = []
            for index, movie in enumerate(data):
                for key, value in movie.items():
                    if value in [None, '', 'N/A']:
                        missing_values.append((index, key))
            
            if missing_values:
                print(f"Các giá trị thiếu (index, key): {missing_values}")
            else:
                print("Không có giá trị thiếu.")

    elif isinstance(data, dict):
        print(f"Số lượng thuộc tính trong từ điển: {len(data)}")
    else:
        print("Dữ liệu không phải là danh sách hoặc từ điển.")



In [7]:
check_size(file_name='movies_data_batch_2.json')

Số lượng phần tử trong danh sách: 7600
Kích thước của phần tử đầu tiên: 2 thuộc tính
Các Movie ID trùng lặp: {'tt33481094', 'tt2250912', 'tt1436562', 'tt0485947', 'tt0057115', 'tt0309593', 'tt1979376', 'tt2555736', 'tt2494362', 'tt2058673', 'tt10720352', 'tt3289956', 'tt0780521', 'tt1025100', 'tt0107889', 'tt0120804', 'tt22408160', 'tt9484998', 'tt0172495', 'tt2056771', 'tt12349832', 'tt31556612', 'tt14253846', 'tt0149261', 'tt14138650', 'tt13457790', 'tt0903624', 'tt0097493', 'tt0120611', 'tt5112584', 'tt3521126', 'tt23218054', 'tt0265208', 'tt1179933', 'tt1538403', 'tt0111282', 'tt27665778', 'tt0095925', 'tt0119698', 'tt0066549', 'tt0098354', 'tt0458339', 'tt0119177', 'tt0097162', 'tt0088846', 'tt2771200', 'tt1524930', 'tt6566576', 'tt0078902', 'tt10288566', 'tt0120812', 'tt7983904', 'tt1285016', 'tt0322330', 'tt12930350', 'tt0076786', 'tt8695030', 'tt0092675', 'tt4364194', 'tt4332232', 'tt0111503', 'tt0316356', 'tt2679042', 'tt0054331', 'tt5478478', 'tt5022702', 'tt0077289', 'tt0359

In [1]:
import os
import json

def remove_duplicates(file_name, output_file_name):
    # Đọc file JSON
    with open(os.path.join(os.getcwd(), file_name), 'r', encoding='utf-8') as file:
        data = json.load(file)

    if isinstance(data, list):
        print(f"Số lượng phần tử ban đầu: {len(data)}")
        
        # Sử dụng set để lưu trữ các Movie ID đã xuất hiện
        seen_movie_ids = set()
        unique_data = []

        for movie in data:
            movie_id = movie.get('Movie ID')
            if movie_id and movie_id not in seen_movie_ids:
                unique_data.append(movie)  # Thêm movie vào danh sách nếu chưa thấy Movie ID trước đó
                seen_movie_ids.add(movie_id)
        
        print(f"Số lượng phần tử sau khi xoá trùng lặp: {len(unique_data)}")

        # Lưu file đã làm sạch
        with open(output_file_name, 'w', encoding='utf-8') as output_file:
            json.dump(unique_data, output_file, ensure_ascii=False, indent=4)
        print(f"Dữ liệu đã được lưu vào file {output_file_name}")

    else:
        print("Dữ liệu không phải là danh sách hoặc không thể xử lý trùng lặp.")



In [2]:

# Ví dụ sử dụng
remove_duplicates('movies_data_batch_4.json', 'movies_data_cleaned.json')

Số lượng phần tử ban đầu: 25200
Số lượng phần tử sau khi xoá trùng lặp: 10050
Dữ liệu đã được lưu vào file movies_data_cleaned.json


## Scraping Reviews of each Movies

In [20]:
# ReviewsScraper class to fetch reviews for each movie
class MovieReviewScraper(BaseScraper):
    def __init__(self, json_file_path):
        super().__init__()  # Call the base class constructor
        self.json_file_path = json_file_path  # Path to the JSON file
        self.movie_data = self._load_movies_from_json()  # Load movie data from JSON
        self.movie_reviews = []  # Adjusted to be a list of movie objects
        self.clicks = 0  # Initialize click counter

    def _load_movies_from_json(self):
        with open(self.json_file_path, 'r', encoding='utf-8') as f:
            return json.load(f)  # Load JSON data

    def fetch_reviews(self):
        for movie in self.movie_data:  # Iterate through the list of movies
            movie_id = movie['Movie ID']
            title = movie['Title']
            review_url = f"https://www.imdb.com/title/{movie_id}/reviews"
            self.driver.get(review_url)

            self._load_reviews()

            wait_time = self._calculate_wait_time(10, self.clicks)  # Adjust wait time based on click count
            time.sleep(wait_time)

            html = self.driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            self._extract_reviews(soup, movie_id, title)

        self.close_driver()
        self.movie_reviews

    def _load_reviews(self):
        # Try to find and click the 'All' reviews button
        try:
            all_reviews_button = WebDriverWait(self.driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='__next']/main/div/section/div/section/div/div[1]/section[1]/div[3]/div/span[2]/button"))
            )
            self.driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", all_reviews_button)
            time.sleep(2)  # Wait for the page to load if needed
            all_reviews_button.click()
        except Exception as e:
            print("Could not find 'All' button, will try to find 'Load More' button.")
            self._load_more_reviews()

    def _load_more_reviews(self):
        # Add progress bar for loading more reviews
        with tqdm(total=10, desc='Loading More Reviews', leave=False) as pbar:
            while True:
                try:
                    load_more_button = WebDriverWait(self.driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="load-more-trigger"]'))
                    )
                    self.driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
                    load_more_button.click()

                    self.clicks += 1  # Increment the click count
                    pbar.update(1)  # Update progress bar

                    wait_time = self._calculate_wait_time(1, self.clicks)  # Adjust wait time based on click count
                    time.sleep(wait_time)

                except Exception as e:
                    print("No more 'Load More' buttons to click.")
                    break

    def _calculate_wait_time(self, base_wait_time, clicks):
        """
        Calculate an adaptive wait time based on the number of clicks.
        As the number of clicks increases, the wait time grows exponentially to accommodate website lag.
        """
        growth_factor = 1.2  # Exponential growth factor
        additional_wait_time = base_wait_time * (growth_factor ** (clicks // 10))  # Increase wait time every 10 clicks
        
        return base_wait_time + additional_wait_time


    def _extract_reviews(self, soup, movie_id, title):
        reviews = soup.select('article.user-review-item')  # Attempt to extract reviews using one selector
        movie_info = {
            'Movie ID': movie_id,
            'Reviews': []
        }
        # If no reviews found, try to load more reviews
        if not reviews:  
            reviews = soup.select('div.lister-item.mode-detail.imdb-user-review')
            if not reviews: # If still no reviews available
                print(f"No reviews found for {title}.")
                return

            for review in reviews:
                parsed_review = self._parse_review(review, "load_more")
                movie_info['Reviews'].append(parsed_review)
        else: # If "all" button found
            for review in reviews:
                parsed_review = self._parse_review(review, "all")
                movie_info['Reviews'].append(parsed_review)

        self.movie_reviews.append(movie_info)

        # Count the number of reviews and display it
        num_reviews = len(movie_info['Reviews'])
        print(f"Total number of reviews for '{title}': {num_reviews}")

    def _parse_review(self, review, button_type):
        """
        Extract information from the review and return as a dictionary.
        """
        # Extract information from the review based on its type (load_more or all)
        if button_type == "load_more":
            review_rating = review.select_one('span.rating-other-user-rating span').get_text(strip=True) if review.select_one('span.rating-other-user-rating span') else 'No rating'
            review_summary = review.select_one('a.title').get_text(strip=True) if review.select_one('a.title') else 'No summary'
            review_text = review.select_one('div.text.show-more__control').get_text(strip=True) if review.select_one('div.text.show-more__control') else 'No content'
            author_tag = review.select_one('span.display-name-link a').get_text(strip=True) if review.select_one('span.display-name-link a') else 'Unknown Author'
            review_date = review.select_one('span.review-date').get_text(strip=True) if review.select_one('span.review-date') else 'No date'
        else:
            review_rating = review.select_one('span.ipc-rating-star--rating').get_text(strip=True) if review.select_one('span.ipc-rating-star--rating') else 'No rating'
            review_summary = review.select_one('span[data-testid="review-summary"]').get_text(strip=True) if review.select_one('span[data-testid="review-summary"]') else 'No summary'
            review_text = review.select_one('div.ipc-html-content-inner-div').get_text(strip=True) if review.select_one('div.ipc-html-content-inner-div') else 'No content'
            author_tag = review.select_one('a[data-testid="author-link"]').get_text(strip=True) if review.select_one('a[data-testid="author-link"]') else 'Unknown Author'
            review_date = review.select_one('li.review-date').get_text(strip=True) if review.select_one('li.review-date') else 'No date'

        # Return the review information in the expected format
        return {
            'Review Summary': review_summary,
            'Review': review_text,
            'Rating': review_rating,
            'Author': author_tag,
            'Date': review_date
        }
    def save_to_json(self):
        with open('movies_reviews.json', 'w', encoding='utf-8') as f:
            json.dump(self.movie_reviews, f, ensure_ascii=False, indent=4)
        print("Reviews saved to movies_reviews.json")

### Test 1 movie

#### Test case

In [21]:
import os
#test reviews scraper
data = [
    {
        "Movie ID": "tt17526714",
        "Title": "The Substance"
    }
]

# Save the data to a JSON file
with open('test_movie.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

path = os.path.join(os.getcwd(), 'test_movie.json')
scraper = MovieReviewScraper(path)

scraper.fetch_reviews()
scraper.save_to_json()
print("Movies fetched and saved to movies_reviews.json")


Could not find 'All' button, will try to find 'Load More' button.


No more 'Load More' buttons to click.


Total number of reviews for 'The Substance': 433
Reviews saved to movies_reviews.json
Movies fetched and saved to movies_reviews.json


In [26]:
import json
import os

# Đọc file JSON với mã hóa utf-8
with open(os.path.join(os.getcwd(), 'movies_reviews.json'), 'r', encoding='utf-8') as file:
    data = json.load(file)

# Đếm số lượng review cho từng phim
for movie in data:
    movie_id = movie["Movie ID"]
    reviews = movie["Reviews"]
    review_count = len(reviews)
    print(f"Movie ID: {movie_id} có {review_count} review(s).")


Movie ID: tt17526714 có 433 review(s).
